## Imports

In [46]:
import pandas as pd
from keras.models import Sequential
from keras.models import clone_model
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import numpy as np
import random

## Read data

In [47]:
df=pd.read_excel("Delays_W_L_FAMIRRORSimulated_50K.xlsx")
dataset=df.values
df.head(10)

,wp1,lp1,wp2,lp2,wp3,lp3,wp4,lp4,wp5,lp5,wp6,lp6,wp7,lp7,wp8,lp8,wp9,lp9,wp10,lp10,wp11,lp11,wp12,lp12,wp13,lp13,wp14,lp14,wn1,ln1,wn2,ln2,wn3,ln3,wn4,ln4,wn5,ln5,wn6,ln6,wn7,ln7,wn8,ln8,wn9,ln9,wn10,ln10,wn11,ln11,wn12,ln12,wn13,ln13,wn14,ln14,delay_lh_nodeaco,delay_hl_nodeaco,delay_lh_nodebco,delay_hl_nodebco,delay_hl_nodecco,delay_lh_nodecco
0,1.050000e-07,2.300000e-08,7.100000e-08,2.400000e-08,8.000000e-08,2.800000e-08,9.500000e-08,2.300000e-08,4.600000e-08,2.300000e-08,6.100000e-08,2.300000e-08,4.900000e-08,3.200000e-08,1.100000e-07,3.100000e-08,9.700000e-08,2.200000e-08,9.200000e-08,2.700000e-08,8.500000e-08,2.400000e-08,8.500000e-08,3.200000e-08,1.180000e-07,2.500000e-08,5.400000e-08,2.600000e-08,6.200000e-08,2.300000e-08,6.100000e-08,2.300000e-08,3.700000e-08,2.400000e-08,8.400000e-08,2.900000e-08,4.800000e-08,2.900000e-08,6.500000e-08,2.600000e-08,9.600000e-08,3.100000e-08,8.600000e-08,2.700000e-08,3.100000e-08,3.200000e-08,2.900000e-08,3.000000e-08,2.300000e-08,3.100000e-08,3.000000e-08,3.000000e-08,4.600000e-08,2.600000e-08,1.110000e-07,2.600000e-08,2.522120e-11,3.173040e-11,2.226566e-11,3.462740e-11,3.342622e-11,2.337032e-11
1,7.400000e-08,2.700000e-08,8.500000e-08,3.100000e-08,4.100000e-08,3.000000e-08,9.800000e-08,3.000000e-08,1.020000e-07,2.300000e-08,1.170000e-07,2.700000e-08,4.600000e-08,3.200000e-08,3.500000e-08,3.100000e-08,6.900000e-08,2.900000e-08,1.080000e-07,2.400000e-08,5.600000e-08,2.900000e-08,1.090000e-07,3.100000e-08,5.600000e-08,2.900000e-08,7.300000e-08,2.300000e-08,1.040000e-07,2.400000e-08,9.500000e-08,3.100000e-08,5.500000e-08,2.800000e-08,8.400000e-08,2.200000e-08,1.090000e-07,2.800000e-08,2.200000e-08,2.800000e-08,7.900000e-08,2.400000e-08,1.200000e-07,2.600000e-08,2.900000e-08,2.200000e-08,8.200000e-08,2.600000e-08,1.170000e-07,3.200000e-08,6.900000e-08,2.500000e-08,7.600000e-08,2.400000e-08,3.600000e-08,2.500000e-08,5.238899e-11,3.799978e-11,5.539780e-11,2.192646e-11,3.488078e-11,5.494923e-11
2,6.500000e-08,2.800000e-08,1.100000e-07,2.300000e-08,2.400000e-08,3.000000e-08,9.400000e-08,3.000000e-08,7.000000e-08,2.300000e-08,9.300000e-08,2.400000e-08,8.800000e-08,2.800000e-08,4.100000e-08,2.400000e-08,3.800000e-08,2.800000e-08,1.040000e-07,2.600000e-08,6.000000e-08,3.000000e-08,6.500000e-08,3.100000e-08,6.700000e-08,2.600000e-08,2.400000e-08,2.200000e-08,2.600000e-08,2.600000e-08,8.100000e-08,3.200000e-08,6.400000e-08,2.200000e-08,5.700000e-08,2.700000e-08,5.100000e-08,2.600000e-08,5.700000e-08,3.000000e-08,1.110000e-07,2.600000e-08,3.100000e-08,3.200000e-08,6.900000e-08,2.500000e-08,1.060000e-07,2.700000e-08,9.700000e-08,2.900000e-08,5.000000e-08,2.200000e-08,1.050000e-07,2.800000e-08,1.030000e-07,3.000000e-08,3.960059e-11,5.370349e-11,3.837178e-11,2.275030e-11,4.715514e-11,4.289461e-11
3,3.500000e-08,3.200000e-08,3.400000e-08,2.700000e-08,5.000000e-08,2.700000e-08,5.800000e-08,2.700000e-08,7.600000e-08,3.000000e-08,1.030000e-07,2.300000e-08,4.800000e-08,2.200000e-08,7.600000e-08,2.200000e-08,1.120000e-07,3.100000e-08,1.040000e-07,2.900000e-08,1.130000e-07,2.200000e-08,5.900000e-08,2.700000e-08,2.600000e-08,2.200000e-08,1.130000e-07,2.800000e-08,6.600000e-08,2.400000e-08,7.900000e-08,2.900000e-08,1.100000e-07,2.300000e-08,8.700000e-08,3.200000e-08,6.600000e-08,2.600000e-08,4.200000e-08,2.700000e-08,3.400000e-08,2.900000e-08,7.600000e-08,2.700000e-08,6.700000e-08,2.500000e-08,9.300000e-08,2.600000e-08,7.800000e-08,2.300000e-08,1.040000e-07,2.600000e-08,1.200000e-07,3.200000e-08,6.600000e-08,2.400000e-08,7.751387e-11,6.390787e-11,7.703473e-11,4.169431e-11,4.903933e-11,7.545821e-11
4,9.500000e-08,2.700000e-08,1.030000e-07,2.800000e-08,1.070000e-07,2.800000e-08,2.900000e-08,2.700000e-08,5.700000e-08,2.200000e-08,9.300000e-08,2.600000e-08,5.000000e-08,2.900000e-08,1.070000e-07,3.000000e-08,3.800000e-08,2.200000e-08,8.200000e-08,2.600000e-08,1.180000e-07,2.700000e-08,1.150000e-07,3.100000e-08,4.000000e-08,3.100000e-08,7.500000e-08,2.900000e-08,8.600000e-08,2.800000e-08,3.200000e-08,2.900000e-08,7.6

## Training and testing model

In [48]:
X = dataset[:40000,0:56]
y = dataset[:40000,56:]

In [49]:
scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
scalarX.fit(X)
scalarY.fit(y)
X = scalarX.transform(X)
y = scalarY.transform(y)

In [50]:
model = Sequential()
model.add(Dense(43, input_dim=56, activation='relu'))
model.add(Dense(35, activation='relu'))
model.add(Dense(29, activation='relu'))
model.add(Dense(6, activation='linear'))
model.compile(loss='mse', optimizer='adam')
model.fit(X, y, epochs=1000, verbose=1)

Epoch 1/1000
40000/40000 [==============================] - 2s 42us/step - loss: 0.0076
Epoch 2/1000
40000/40000 [==============================] - 2s 40us/step - loss: 0.0020
Epoch 3/1000
40000/40000 [==============================] - 2s 40us/step - loss: 0.0017
Epoch 4/1000
40000/40000 [==============================] - 2s 39us/step - loss: 0.0013
Epoch 5/1000
40000/40000 [==============================] - 2s 40us/step - loss: 0.0010
Epoch 6/1000
40000/40000 [==============================] - 2s 39us/step - loss: 6.6636e-04
Epoch 7/1000
40000/40000 [==============================] - 2s 39us/step - loss: 5.0825e-04
Epoch 8/1000
40000/40000 [==============================] - 2s 39us/step - loss: 4.0322e-04
Epoch 9/1000
40000/40000 [==============================] - 2s 39us/step - loss: 3.5541e-04
Epoch 10/1000
40000/40000 [==============================] - 2s 39us/step - loss: 3.3183e-04
Epoch 11/1000
40000/40000 [==============================] - 2s 40us/step - loss: 3.1339e-04
Epoch 

In [51]:
X_test = dataset[40000:,0:56]
y_test = dataset[40000:,56:]
X_test = scalarX.transform(X_test)
y_test = scalarY.transform(y_test)

y_trainpred = model.predict(X)
y_trainpred2 = scalarY.inverse_transform(y_trainpred)

y_testpred = model.predict(X_test)
y_testpred2 = scalarY.inverse_transform(y_testpred)

y_test2 = scalarY.inverse_transform(y_test)
y_2 = scalarY.inverse_transform(y)

In [52]:
for i in range(6):
    r2_train = r2_score(y_trainpred2[:,i], y_2[:,i])
    r2_test = r2_score(y_testpred2[:,i], y_test2[:,i])
    print("Train R2 Score for "+df.columns[56+i]+" : "+str(r2_train))
    print("Test  R2 Score for "+df.columns[56+i]+" : "+str(r2_test))
    print()

Train R2 Score for delay_lh_nodeaco : 0.9994046608724173
Test  R2 Score for delay_lh_nodeaco : 0.999374819392381

Train R2 Score for delay_hl_nodeaco : 0.9948789645269915
Test  R2 Score for delay_hl_nodeaco : 0.994608010649042

Train R2 Score for delay_lh_nodebco : 0.9990192578223169
Test  R2 Score for delay_lh_nodebco : 0.998986460457769

Train R2 Score for delay_hl_nodebco : 0.9948383190142497
Test  R2 Score for delay_hl_nodebco : 0.9945178910300352

Train R2 Score for delay_hl_nodecco : 0.9946898030607043
Test  R2 Score for delay_hl_nodecco : 0.994517770781492

Train R2 Score for delay_lh_nodecco : 0.9991052820371301
Test  R2 Score for delay_lh_nodecco : 0.9990756731301712



In [53]:
for i in range(6):
    mse_train = mean_squared_error(y_trainpred2[:,i], y_2[:,i])
    mse_test = mean_squared_error(y_testpred2[:,i], y_test2[:,i])
    print("Train MSE Score "+df.columns[56+i]+" : "+str(mse_train))
    print("Test  MSE Score "+df.columns[56+i]+" : "+str(mse_test))
    print()

Train MSE Score delay_lh_nodeaco : 5.272679341647875e-25
Test  MSE Score delay_lh_nodeaco : 5.474696921213461e-25

Train MSE Score delay_hl_nodeaco : 1.4574891692094481e-24
Test  MSE Score delay_hl_nodeaco : 1.5108603850397582e-24

Train MSE Score delay_lh_nodebco : 8.730812839653967e-25
Test  MSE Score delay_lh_nodebco : 8.897702315490419e-25

Train MSE Score delay_hl_nodebco : 1.5977141940823207e-24
Test  MSE Score delay_hl_nodebco : 1.7398940413935626e-24

Train MSE Score delay_hl_nodecco : 1.3482414184121719e-24
Test  MSE Score delay_hl_nodecco : 1.3987928147107023e-24

Train MSE Score delay_lh_nodecco : 7.9035149277989815e-25
Test  MSE Score delay_lh_nodecco : 8.081017310651024e-25



In [54]:
for i in range(len(X_test)):
    print(y_test2[i, :])
    print(y_testpred2[i, :])    
    print()

Streaming output truncated to the last 5000 lines.
 2.90799917e-11 6.08662743e-11]
[6.4389293e-11 6.1084027e-11 6.2016711e-11 2.6450659e-11 3.0221853e-11
 6.1037161e-11]

[4.17988014e-11 2.68821870e-11 3.51893011e-11 2.32055461e-11
 1.99249212e-11 3.86727751e-11]
[4.1959065e-11 2.6973854e-11 3.4405128e-11 2.3590669e-11 2.0317805e-11
 3.8554479e-11]

[5.53396489e-11 5.28586192e-11 5.79010670e-11 3.18110411e-11
 4.29857415e-11 6.12334905e-11]
[5.4967694e-11 5.3178965e-11 5.5946990e-11 3.2552398e-11 4.4660390e-11
 6.0093888e-11]

[2.64538406e-11 3.49804006e-11 2.96008528e-11 3.73687486e-11
 6.66776795e-11 2.89876256e-11]
[2.6692482e-11 3.4205791e-11 2.8553002e-11 3.7526437e-11 6.6579464e-11
 2.7319121e-11]

[3.98155931e-11 2.94431592e-11 3.96594536e-11 3.60363119e-11
 2.23047597e-11 4.01405881e-11]
[3.9492479e-11 2.9661024e-11 3.9251907e-11 3.7225462e-11 2.2430245e-11
 3.9495133e-11]

[2.83996819e-11 4.41929131e-11 2.85078601e-11 4.35858885e-11
 3.66532322e-11 2.61456117e-11]
[2.8926830e-

## Converging to minima using genetic algorithm

In [88]:
seed_data = dataset[0,0:56].tolist()

GENE_LEN = len(seed_data)
ULIM_COEFF = 0.001
LLIM_COEFF = 0.0
POPULATION_SIZE = 10 # keep it even
step = 10**(-1)
ALLOWED_ACCESS = 100000 # access to model
ELITE_COUNT = int( POPULATION_SIZE * 0.2)
CROSS_COUNT = POPULATION_SIZE - ELITE_COUNT

In [56]:
def give_order(num):
	'''
	returns the order of magnitude of number given
	'''
	num = abs(num)

	if num == 0.0:
		order = 0
	elif num >= 1.0:
		n = int(num)
		order = -1
		while n > 0:
			n = int(n/10)
			order += 1
	else:
		order = 0
		while num < 1.0:
			num *= 10
			order -= 1
	return order

In [57]:
def mutate(num):
	'''
	num is number to mutate
	step is magnitude by which to mutate
	'''
	r = (random.random() - 0.5) * 2.0 # generates number [-1.0, 1.0]
	r *= step * 10**give_order(num)
	r += num

	if r > ULIM_COEFF:
		return ULIM_COEFF
	elif r < LLIM_COEFF:
		return LLIM_COEFF
	else:
		return r

In [58]:
def calculate_fitness(x):
	'''
	Lower fitness value => more fit.
	Fittest at value 0.
	'''
	fitness_weights = np.array([+1.0, +1.0, +1.0, +1.0, +1.0, +1.0])
  
	x = np.array(x)
	x = np.expand_dims(x, axis=0)
	x = scalarX.transform(x)
	y_pred = scalarY.inverse_transform(model.predict(x))
	
	return (np.sum(y_pred * fitness_weights)).item()

In [59]:
def calculate_rank(f):
	'''
	f is fitness
	sf is sorted fitness
	r contains ranks of f
	'''
	sf = sorted(f)
	r = [] # ranks
	for i in f:
		rnum = sf.index(i)
		while rnum in r: # To avoid duplicate ranks
			rnum += 1
		r.append(rnum)
	return r

In [60]:
def initial_population():
	'''
	p is population
	'''
	mutation_probability = 0.4 # TUNE
	p = []
	p.append(seed_data)
	for i in range(POPULATION_SIZE-1):
		gene = seed_data[:]
		for i in range(GENE_LEN):
			if random.random() < mutation_probability:
				gene[i] = mutate(gene[i])
		p.append(gene)
	return p

In [61]:
def calculate_chance(r, p):
	'''
	r is rank
	pop is population
	'''
	pc = 0.30
	c = []
	for i in range(POPULATION_SIZE):
		c.append(pc * (1-pc)**r[i])
	return c 

In [62]:
def select_population(pop, c):
	'''
	pop is population
	c is chance
	'''
	return random.choices(pop, c, k=CROSS_COUNT)

In [63]:
def do_crossover(s):
	'''
	s is selected population
	co is crossover
	p parent
	c child
	'''
	co = []
	for i in range(CROSS_COUNT):
		p1 = s[(i+0) % CROSS_COUNT]
		p2 = s[(i+1) % CROSS_COUNT]
		c = []
		for j in range(GENE_LEN):
			if random.random() < 0.5:
				c.append(p1[j])
			else:
				c.append(p2[j])
		co.append(c)

	return co

In [77]:
def do_mutation(c):
	'''
	c is cross
	mc is mutated cross
	mg is mutated gene
	'''
	mutation_probability = 0.15

	mc = []
	for gene in c:
		mg = []
		for i in gene:
			if random.random() <= mutation_probability:
				mg.append(mutate(i))
			else:
				mg.append(i)
		mc.append(mg)

	global step
	step *= 0.9999 # keep less than 1.0
	
	return mc

In [65]:
def pick_elite(population, rank):
	elite = []
	cur_rank = 0 # to avoid duplicates
	for i in range(ELITE_COUNT):
		elite.append(population[rank.index(cur_rank)])
		cur_rank += 1
		while(cur_rank < len(population) and population[rank.index(cur_rank)] in elite):
			cur_rank += 1
		if cur_rank >= len(population):
			cur_rank = 0 
	return elite

In [66]:
def print_delays(x):
    '''
    prints the delay of optimized l and w
    '''
    x = np.array(x)
    x = np.expand_dims(x, axis=0)
    x = scalarX.transform(x)
    y_pred = scalarY.inverse_transform(model.predict(x))

    print('optimised delays:', y_pred)

In [89]:
fitest = [float('inf'), []] # [fitness_value, genes]
population = initial_population()

generation = 1
access_left = ALLOWED_ACCESS
while access_left > POPULATION_SIZE:
	fitness = []
	for i in population:
		fitness.append(calculate_fitness(i))

	rank = calculate_rank(fitness[:])	
	
	# print(fitness[rank.index(0)])
	if fitness[rank.index(0)] < fitest[0]:
		fitest[0] = fitness[rank.index(0)]
		fitest[1] = population[rank.index(0)][:]
		print('generation =', generation, ' fitness =', fitest[0])

	chance = calculate_chance(rank, population)
	select = select_population(population, chance)
	cross = do_crossover(select)
	elite = pick_elite(population, rank)
	population = do_mutation(cross) + elite
	
	access_left -= POPULATION_SIZE
	generation += 1

print('optimised input variables:', fitest[1])
print_delays(fitest[1])

Streaming output truncated to the last 5000 lines.
generation = 1080  fitness = 7.741841542840966e-11
generation = 1082  fitness = 7.736127016766403e-11
generation = 1084  fitness = 7.734285087379611e-11
generation = 1085  fitness = 7.733761894779256e-11
generation = 1086  fitness = 7.732002885174616e-11
generation = 1088  fitness = 7.717768785164836e-11
generation = 1094  fitness = 7.71499704399492e-11
generation = 1096  fitness = 7.714485994458897e-11
generation = 1097  fitness = 7.713790023400335e-11
generation = 1098  fitness = 7.710443741815176e-11
generation = 1099  fitness = 7.710032959296065e-11
generation = 1100  fitness = 7.706449367539392e-11
generation = 1101  fitness = 7.697488479951886e-11
generation = 1102  fitness = 7.694626186216524e-11
generation = 1103  fitness = 7.688140749029237e-11
generation = 1105  fitness = 7.687087424934624e-11
generation = 1108  fitness = 7.686052141964161e-11
generation = 1109  fitness = 7.684526279194692e-11
generation = 1110  fitness = 7.6